In [1]:
:!./install.sh

# 인터프리터에서 변수, 조건문 다루기
TODO

In [2]:
data Expr = Var String
          | Val Int
          | Add Expr Expr
          -- | Sub Expr Expr
          -- | Mul Expr Expr
          -- | Div Expr Expr
          | If Expr Expr Expr
          deriving Show

type Env = [ (String, Int) ]

eval :: Env -> Expr -> Int
eval env (Var x) = case lookup x env of
                     Nothing -> error (x ++ " not found") 
                     Just n  -> n
eval env (Val n) = n
eval env (Add e1 e2) = eval env e1 + eval env e2
eval env (If e e1 e0) = case eval env e of
                          0 -> eval env e0
                          _ -> eval env e1

In [4]:
{-  b = ?,  x = 2, y = 4

    if b then (x + 3) else y  -}

e = If (Var "b") (Add (Var "x") (Val 3)) (Var "y")

eval [("x",2), ("y",4), ("b",0)] e
eval [("x",2), ("y",4), ("b",1)] e  
eval [("x",2), ("y",4)]          e -- 에러 발생

4

5

: 